# Aggregation and Grouping

An essential piece of analysis of large data is efficient summarization: computing aggregations like ``sum()``, ``mean()``, ``median()``, ``min()``, and ``max()``, in which a single number gives insight into the nature of a potentially large dataset.
In this section, we'll explore aggregations in Pandas, from simple operations akin to what we've seen on NumPy arrays, to more sophisticated operations based on the concept of a ``groupby``.

For convenience, we'll use the ``display`` magic function:

In [42]:
import matplotlib.pyplot as plt #import the matplotlib
%matplotlib inline 
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()

In [43]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Simple Aggregation in Pandas

Earlier, we explored some of the data aggregations available for NumPy arrays (["Aggregations: Min, Max, and Everything In Between"](02.04-Computation-on-arrays-aggregates.ipynb)).
As with a one-dimensional NumPy array, for a Pandas ``Series`` the aggregates return a single value:

In [44]:
tips = sns.load_dataset('tips')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [45]:
tips.describe(include = 'all')

,total_bill,tip,sex,smoker,day,time,size
count,244.000000,244.000000,244,244,244,244,244.000000
unique,NaN,NaN,2,2,4,2,NaN
top,NaN,NaN,Male,No,Sat,Dinner,NaN
freq,NaN,NaN,157,151,87,176,NaN
mean,19.785943,2.998279,NaN,NaN,NaN,NaN,2.569672
std,8.902412,1.383638,NaN,NaN,NaN,NaN,0.951100
min,3.070000,1.000000,NaN,NaN,NaN,NaN,1.000000
25%,13.347500,2.000000,NaN,NaN,NaN,NaN,2.000000
50%,17.795000,2.900000,NaN,NaN,NaN,NaN,2.000000
75%,24.127500,3.562500,NaN,NaN,NaN,NaN,3.000000


In [46]:
tips.dropna().describe()
# dropna() : used to remove missing values

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [47]:
tips['total_bill'].mean()

19.785942622950824

The following table summarizes some other built-in Pandas aggregations:

| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | Total number of items           |
| ``first()``, ``last()``  | First and last item             |
| ``mean()``, ``median()`` | Mean and median                 |
| ``min()``, ``max()``     | Minimum and maximum             |
| ``std()``, ``var()``     | Standard deviation and variance |
| ``mad()``                | Mean absolute deviation         |
| ``prod()``               | Product of all items            |
| ``sum()``                | Sum of all items                |

These are all methods of ``DataFrame`` and ``Series`` objects.

To go deeper into the data, however, simple aggregates are often not enough.
The next level of data summarization is the ``groupby`` operation, which allows you to quickly and efficiently compute aggregates on subsets of data.

## GroupBy: Split, Apply, Combine

Simple aggregations can give you a flavor of your dataset, but often we would prefer to aggregate conditionally on some label or index: this is implemented in the so-called ``groupby`` operation.
The name "group by" comes from a command in the SQL database language, but it is perhaps more illuminative to think of it in the terms first coined by Hadley Wickham of Rstats fame: *split, apply, combine*.

### Split, apply, combine

A canonical example of this split-apply-combine operation, where the "apply" is a summation aggregation, is illustrated in this figure:

![](figures/[04-02]_Fig_04_01_split-apply-combine.png) 

This makes clear what the ``groupby`` accomplishes:

- The *split* step involves breaking up and grouping a ``DataFrame`` depending on the value of the specified key.
- The *apply* step involves computing some function, usually an aggregate, transformation, or filtering, within the individual groups.
- The *combine* step merges the results of these operations into an output array.

While this could certainly be done manually using some combination of the masking, aggregation, and merging commands covered earlier, an important realization is that *the intermediate splits do not need to be explicitly instantiated*. Rather, the ``GroupBy`` can (often) do this in a single pass over the data, updating the sum, mean, count, min, or other aggregate for each group along the way.
The power of the ``GroupBy`` is that it abstracts away these steps: the user need not think about *how* the computation is done under the hood, but rather thinks about the *operation as a whole*.

As a concrete example, let's take a look at using Pandas for the computation shown in this diagram.
We'll start by creating the input ``DataFrame``:

In [48]:
tips.groupby("sex")

Notice that what is returned is not a set of ``DataFrame``s, but a ``DataFrameGroupBy`` object.
This object is where the magic is: you can think of it as a special view of the ``DataFrame``, which is poised to dig into the groups but does no actual computation until the aggregation is applied.
This "lazy evaluation" approach means that common aggregates can be implemented very efficiently in a way that is almost transparent to the user.

To produce a result, we can apply an aggregate to this ``DataFrameGroupBy`` object, which will perform the appropriate apply/combine steps to produce the desired result:

In [49]:
tips.groupby("sex").sum()
# work only for the numerical attribute

,total_bill,tip,size
sex,,,
Male,3256.82,485.07,413
Female,1570.95,246.51,214


In [50]:
tips.groupby("sex").mean()
# work only for the numerical attribute

,total_bill,tip,size
sex,,,
Male,20.744076,3.089618,2.630573
Female,18.056897,2.833448,2.459770


In [51]:
tips.groupby("sex").median()

,total_bill,tip,size
sex,,,
Male,18.35,3.00,2
Female,16.40,2.75,2


The ``sum()`` method is just one possibility here; you can apply virtually any common Pandas or NumPy aggregation function, as well as virtually any valid ``DataFrame`` operation, as we will see in the following discussion.

### The GroupBy object

The ``GroupBy`` object is a very flexible abstraction.
In many ways, you can simply treat it as if it's a collection of ``DataFrame``s, and it does the difficult things under the hood. Let's see some examples using the Planets data.

Perhaps the most important operations made available by a ``GroupBy`` are *aggregate*, *filter*, *transform*, and *apply*.
We'll discuss each of these more fully in ["Aggregate, Filter, Transform, Apply"](#Aggregate,-Filter,-Transform,-Apply), but before that let's introduce some of the other functionality that can be used with the basic ``GroupBy`` operation.

#### Column indexing

The ``GroupBy`` object supports column indexing in the same way as the ``DataFrame``, and returns a modified ``GroupBy`` object.
For example:

In [52]:
tips.groupby('sex')

In [53]:
tips.groupby('sex')['total_bill']

Here we've selected a particular ``Series`` group from the original ``DataFrame`` group by reference to its column name.
As with the ``GroupBy`` object, no computation is done until we call some aggregate on the object:

In [54]:
tips.groupby('sex')['total_bill'].median()

sex
Male      18.35
Female    16.40
Name: total_bill, dtype: float64

This gives an idea of the general scale of orbital periods (in days) that each method is sensitive to.

#### Iteration over groups

The ``GroupBy`` object supports direct iteration over the groups, returning each group as a ``Series`` or ``DataFrame``:

In [55]:
for (method, group) in tips.groupby('day'):
    print("{0:20s} shape={1}".format(method, group.shape))
# shape : row & colummn
# 20s : 20 spaces
# shape 1 : 'day' 하나의 종류
# (62, 7) : (row, colummn)

Thur                 shape=(62, 7)
Fri                  shape=(19, 7)
Sat                  shape=(87, 7)
Sun                  shape=(76, 7)


This can be useful for doing certain things manually, though it is often much faster to use the built-in ``apply`` functionality, which we will discuss momentarily.

#### Dispatch methods

Through some Python class magic, any method not explicitly implemented by the ``GroupBy`` object will be passed through and called on the groups, whether they are ``DataFrame`` or ``Series`` objects.
For example, you can use the ``describe()`` method of ``DataFrame``s to perform a set of aggregations that describe each group in the data:

In [56]:
tips.groupby('day')['total_bill'].describe()

,count,mean,std,min,25%,50%,75%,max
day,,,,,,,,
Thur,62.0,17.682742,7.886170,7.51,12.4425,16.20,20.1550,43.11
Fri,19.0,17.151579,8.302660,5.75,12.0950,15.38,21.7500,40.17
Sat,87.0,20.441379,9.480419,3.07,13.9050,18.24,24.7400,50.81
Sun,76.0,21.410000,8.832122,7.25,14.9875,19.63,25.5975,48.17


Looking at this table helps us to better understand the data: for example, the vast majority of planets have been discovered by the Radial Velocity and Transit methods, though the latter only became common (due to new, more accurate telescopes) in the last decade.
The newest methods seem to be Transit Timing Variation and Orbital Brightness Modulation, which were not used to discover a new planet until 2011.

This is just one example of the utility of dispatch methods.
Notice that they are applied *to each individual group*, and the results are then combined within ``GroupBy`` and returned.
Again, any valid ``DataFrame``/``Series`` method can be used on the corresponding ``GroupBy`` object, which allows for some very flexible and powerful operations!

### Aggregate, filter, transform, apply

The preceding discussion focused on aggregation for the combine operation, but there are more options available.
In particular, ``GroupBy`` objects have ``aggregate()``, ``filter()``, ``transform()``, and ``apply()`` methods that efficiently implement a variety of useful operations before combining the grouped data.

For the purpose of the following subsections, we'll use this `tips` ``DataFrame``:

#### Aggregation

We're now familiar with ``GroupBy`` aggregations with ``sum()``, ``median()``, and the like, but the ``aggregate()`` method allows for even more flexibility.
It can take a string, a function, or a list thereof, and compute all the aggregates at once.
Here is a quick example combining all these:

In [57]:
tips.groupby('day').aggregate(['min', np.median, max])

total_bill                 tip               size           
            min median    max   min median    max  min median max
day                                                              
Thur       7.51  16.20  43.11  1.25  2.305   6.70    1      2   6
Fri        5.75  15.38  40.17  1.00  3.000   4.73    1      2   4
Sat        3.07  18.24  50.81  1.00  2.750  10.00    1      2   5
Sun        7.25  19.63  48.17  1.01  3.150   6.50    2      2   6

Another useful pattern is to pass a dictionary mapping column names to operations to be applied on that column:

In [58]:
#Finding the minimum total bill from the maximum size
tips.groupby('day').aggregate({'total_bill': 'min',
                             'size': 'max'})

,total_bill,size
day,,
Thur,7.51,6
Fri,5.75,4
Sat,3.07,5
Sun,7.25,6


In [59]:
tips.groupby('day').aggregate({'total_bill': 'max',
                             'size': 'min'})

,total_bill,size
day,,
Thur,43.11,1
Fri,40.17,1
Sat,50.81,1
Sun,48.17,2


#### Filtering

A filtering operation allows you to drop data based on the group properties.
For example, we might want to keep all groups in which the standard deviation is larger than some critical value:

In [27]:
def filter_func(x):
    return x['total_bill'].std() > 9

display('tips.head()', "tips.groupby('day').std()", "tips.groupby('day').filter(filter_func)")

tips.head()
   total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4

tips.groupby('day').std()
      total_bill       tip      size
day                                 
Thur    7.886170  1.240223  1.066285
Fri     8.302660  1.019577  0.567131
Sat     9.480419  1.631014  0.819275
Sun     8.832122  1.234880  1.007341

tips.groupby('day').filter(filter_func)
     total_bill   tip     sex smoker  day    time  size
19        20.65  3.35    Male     No  Sat  Dinner     3
20        17.92  4.08    Male     No  Sat  Dinner     2
21        20.29  2.75  Female     No  Sat  Dinner     2
22        15.77  2.23  Female     No  Sat  Dinner     2
23        39.42  7.58    Male     No  Sat  Dinner     4
..          ...   ...     ...    ...  ...     ...   ...
238       35.83  4.67  Female     No  Sat  Dinner     3
239       29.03  5.92    Male     No  Sat  Dinner     3
240       27.18  2.00  Female    Yes  Sat  Dinner     2
241       22.67  2.00    Male    Yes  Sat  Dinner     2
242       17.82  1.75    Male     No  Sat  Dinner     2

[87 rows x 7 columns]

The filter function should return a Boolean value specifying whether the group passes the filtering. Here because group A does not have a standard deviation greater than 4, it is dropped from the result.

#### Transformation

While aggregation must return a reduced version of the data, transformation can return some transformed version of the full data to recombine.
For such a transformation, the output is the same shape as the input.
A common example is to center the data by subtracting the group-wise mean:

In [37]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [29]:
tips['total_bill'].mean()

19.785942622950824

In [30]:
tips['total_bill'].head()

0    16.99
1    10.34
2    21.01
3    23.68
4    24.59
Name: total_bill, dtype: float64

In [31]:
tips.groupby('day')['total_bill'].mean()

day
Thur    17.682742
Fri     17.151579
Sat     20.441379
Sun     21.410000
Name: total_bill, dtype: float64

In [33]:
tips.groupby('day')['total_bill'].transform(lambda x: x - x.mean()).head()

0    -4.42
1   -11.07
2    -0.40
3     2.27
4     3.18
Name: total_bill, dtype: float64

In [35]:
tips.groupby('day').transform(lambda x: x - x.mean()).head()
# lamda : function(여기서는 x) ( 이렇게 되는 ) - totalbill, tip, size 모두에 적용
# 16.99 - 21.41 = -4.42
# 10.34 - 21.41 = -11.07
# 21.01 - 21.42 = -0.4
# 21.42 : Sun의 평균을 빼는 것 (x : Sun만 있었기 때문)

,total_bill,tip,size
0,-4.42,-2.245132,-0.842105
1,-11.07,-1.595132,0.157895
2,-0.40,0.244868,0.157895
3,2.27,0.054868,-0.842105
4,3.18,0.354868,1.157895


#### The apply() method

The ``apply()`` method lets you apply an arbitrary function to the group results.
The function should take a ``DataFrame``, and return either a Pandas object (e.g., ``DataFrame``, ``Series``) or a scalar; the combine operation will be tailored to the type of output returned.

For example, here is an ``apply()`` that normalizes the first column by the sum of the second:

In [60]:
tips['tip_percent'] = 100 * tips['tip'] / tips['total_bill']

In [61]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percent
0,16.99,1.01,Female,No,Sun,Dinner,2,5.944673
1,10.34,1.66,Male,No,Sun,Dinner,3,16.054159
2,21.01,3.50,Male,No,Sun,Dinner,3,16.658734
3,23.68,3.31,Male,No,Sun,Dinner,2,13.978041
4,24.59,3.61,Female,No,Sun,Dinner,4,14.680765


In [62]:
tips['data1'] = tips['tip_percent']

In [63]:
def norm_by_data2(x):
    # x is a DataFrame of group values
    x['data1'] /= x['tip_percent'].mean()
    return x

display('tips.head()', "tips.groupby('day').apply(norm_by_data2).head()")
# apply : I want to apply numpy data
# 두번째 table의 data 1 : 첫번째 table의 data1 / tip_percantage의 평균

,total_bill,tip,sex,smoker,day,time,size,tip_percent,data1
0,16.99,1.01,Female,No,Sun,Dinner,2,5.944673,5.944673
1,10.34,1.66,Male,No,Sun,Dinner,3,16.054159,16.054159
2,21.01,3.50,Male,No,Sun,Dinner,3,16.658734,16.658734
3,23.68,3.31,Male,No,Sun,Dinner,2,13.978041,13.978041
4,24.59,3.61,Female,No,Sun,Dinner,4,14.680765,14.680765
,total_bill,tip,sex,smoker,day,time,size,tip_percent,data1
0,16.99,1.01,Female,No,Sun,Dinner,2,5.944673,0.356188
1,10.34,1.66,Male,No,Sun,Dinner,3,16.054159,0.961918
2,21.01,3.50,Male,No,Sun,Dinner,3,16.658734,0.998143
3,23.68,3.31,Male,No,Sun,Dinner,2,13.978041,0.837524


``apply()`` within a ``GroupBy`` is quite flexible: the only criterion is that the function takes a ``DataFrame`` and returns a Pandas object or scalar; what you do in the middle is up to you!

## Summary

This section discusses examples on the use of `groupby` function in Pandas. In particular, the `split-apply-combine` approach is briefly introduced to deal with some basic functions such as aggregate, filtering, transformation, and apply